In [41]:
!pip install speechrecognition pandas googletrans==3.1.0a0 pyttsx3 spacy spacy-langdetect nltk pyaudio
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 11.8 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/12.8 MB 11.7 MB/s eta 0:00:01
     --------------------- ------------------ 6.8/12.8 MB 12.0 MB/s eta 0:00:01
     ---------------------------- ----------- 9.2/12.8 MB 11.9 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 11.8 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 11.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [42]:
!pip install speechrecognition pandas googletrans==3.1.0a0 pyttsx3 spacy spacy-langdetect nltk pyaudio

In [43]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
# Importando Bibliotecas
import speech_recognition as sr
import pandas as pd
from IPython.display import display_markdown, display, HTML as html_print
from googletrans import Translator
import pyttsx3
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from nltk.corpus import stopwords

In [57]:
# Funciones para inicializar SpaCy
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
if not Language.has_factory("language_detector"):
    Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

# Función para intersectar listas
def Intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

# Funciones para imprimir mensajes a colores
def cstr(s, color='black', size='50'):
    return "<text style=color:{};font-size:{}>{}</text>".format(color, size, s)

def print_color(t):
    display(html_print(' '.join([cstr(ti, color=ci, size=si) for ti, ci, si in t])))

# Inicializando variables
r = sr.Recognizer()
r.energy_threshold = 50
translator = Translator()
converter = pyttsx3.init()
converter.setProperty('rate', 200)
converter.setProperty('volume', 1)
converter.setProperty('voice', r'HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0')
lista_fr = stopwords.words('french')
lista_pt = stopwords.words('portuguese')
lista_es = stopwords.words('spanish')
lista_en = stopwords.words('english')

def reconocer(sonido):
    lista = ['fr-FR', 'pt-BR', 'es-ES', 'en-US']
    fr, pt, es, en = 0, 0, 0, 0
    df = pd.DataFrame(columns=['query', 'idioma', 'valor', 'palabras', 'stopwords', 'idioma_final', 'match', 'nota'])
    idioma_final = 'Ninguno'

    for elemento in lista:
        try:
            query = r.recognize_google(sonido, language=elemento)
            valor = nlp(query)._.language.get('score')
            idioma = nlp(query)._.language.get('language')
            palabras = len(query.split())
            stopwords = 0
            if idioma == 'fr': 
                stopwords, fr, idioma_final = len(Intersection(query.split(), lista_fr)), (fr+1), 'Francés'
            if idioma == 'pt': 
                stopwords, pt, idioma_final = len(Intersection(query.split(), lista_pt)), (pt+1), 'Portugués'
            if idioma == 'es': 
                stopwords, es, idioma_final = len(Intersection(query.split(), lista_es)), (es+1), 'Español'
            if idioma == 'en': 
                stopwords, en, idioma_final = len(Intersection(query.split(), lista_en)), (en+1), 'Inglés'
            if idioma == elemento[0:2]: 
                df.loc[len(df)] = [query, idioma, valor, palabras, stopwords, idioma_final, 0, 0.0]
        except Exception as e:
            continue  # Silenciamos errores individuales

    for index, row in df.iterrows():
        if row['idioma'] == 'fr': df.at[index, 'match'] = fr
        if row['idioma'] == 'pt': df.at[index, 'match'] = pt
        if row['idioma'] == 'es': df.at[index, 'match'] = es
        if row['idioma'] == 'en': df.at[index, 'match'] = en

    df['nota'] = (df['palabras'] + df['stopwords']) * df['valor'] * df['match']
    resultado = df[df['nota'] == df['nota'].max()]

    if len(resultado) > 0:
        return resultado['idioma_final'].values[0], resultado['query'].values[0]
    else:
        return 'Ninguno', 'Nada'

def takecommand():
    with sr.Microphone(device_index=0) as source:
        r.adjust_for_ambient_noise(source, duration=1)
        display_markdown('#Escuchando...', raw=True)
        audio = r.listen(source, 0)
    try:
        idioma_detectado, mensaje = reconocer(audio)
        if idioma_detectado != 'Ninguno':
            print_color(((idioma_detectado + ' detectado', 'blue', '40'),))
            print_color((('La persona dijo: ', 'blue', '35'), (mensaje, 'black', '30')))
            text_to_translate = translator.translate(mensaje, dest='en')  # Traducción al inglés como en el código original
            text = text_to_translate.text
            print_color((('La persona dijo: ', 'blue', '35'), (text, 'black', '30')))
            print("\n")
            converter.say(text)
            converter.runAndWait()
            return mensaje
        else:
            print("No se detectó ningún idioma.")
            return "Nada"
    except Exception as e:
        print(e)
        return "Nada"

In [58]:
while True:
    query = takecommand()

#Escuchando...

#Escuchando...

No se detectó ningún idioma.


#Escuchando...

#Escuchando...

#Escuchando...

#Escuchando...

KeyboardInterrupt: 